In [1]:
%reload_ext autoreload
%autoreload 2

In [3]:
import aesara.tensor as aet
from pycmtensor.models import PyCMTensorModel

class ResLogitLayer:
    def __init__(self, u, w_in, w_out, activation=None):
        
        assert w_in.shape[1].eval() == w_out.shape[0].eval()
        assert isinstance(w_in, (Weights)), "w_in must be of type Weights"
        assert isinstance(w_out, (Weights)), "w_out must be of type Weights"
        
        if isinstance(u, (list, tuple)):
            assert len(u) == w_in.shape[0].eval(), f"index.0 of w_in must be of the same length as u"
            self.U = aet.stacklists(u).flatten(2)
        else:
            self.U = u

        self.w_in = w_in()
        self.w_out = w_out()
        if activation == None:
            activation = aet.sigmoid
            
        h = activation(aet.dot(self.U.T, self.w_in))
        output = activation(aet.dot(h, self.w_out)).T
        self.params = [self.w_in, self.w_out]
        self.output = output + self.U

    
    def __repr__(self):
        return f"ResLogitLayer([{self.w_in.shape.eval()}, {self.w_out.shape.eval()}])"

In [4]:
import os
import dill as pickle
import pandas as pd
import aesara.tensor as aet
import pycmtensor as cmt
from pycmtensor.expressions import Beta, Weights
from pycmtensor.optimizers import Adam
from pycmtensor.results import Results
from pycmtensor.models import MNLogit

nb_path = os.path.abspath("")
model_average_sp = pd.read_csv("data/model_average_SP.csv")
model_average_sp.columns = model_average_sp.columns.str.upper()
model_average_sp[["AV1", "AV2"]] = 1
db = cmt.Database("model_average_sp", model_average_sp, choiceVar="CHOICE")
globals().update(db.variables)

# additional steps to format database
db.data['CHOICE'] -= 1
c = db.data.loc[:, 'TT1': 'HH_INC_ABS'].columns.to_list()
db.autoscale(variables=c, verbose=False)

# specify Beta parameters
b_cost = Beta("b_cost", 0.0, None, None, 0)
b_time = Beta("b_time", 0.0, None, None, 0)
b_hw = Beta("b_hw", 0.0, None, None, 0)
b_ch = Beta("b_ch", 0.0, None, None, 0)
b_hh_inc1 = Beta("b_hh_inc1", 0.0, None, None, 0)
b_hh_inc2 = Beta("b_hh_inc2", 0.0, None, None, 1)
b_carav1 = Beta("b_carav1", 0.0, None, None, 0)
b_carav2 = Beta("b_carav2", 0.0, None, None, 1)
b_com = Beta("b_com", 0.0, None, None, 0)
b_shop = Beta("b_shop", 0.0, None, None, 0)
b_bis = Beta("b_bis", 0.0, None, None, 0)
b_lei = Beta("b_lei", 0.0, None, None, 0)
asc_alt1 = Beta("asc_alt1", 0.0, None, None, 1)
asc_alt2 = Beta("asc_alt2", 0.0, None, None, 0)

# specify weight parameters
W1 = Weights("ResNet_01a", (2, 10), 0, True)
W2 = Weights("ResNet_01b", (10, 2), 0, True)

U_1 = (
    b_cost * db["TC1"] + b_time * db["TT1"] + b_hw * db["HW1"] + b_ch * db["CH1"] 
    + b_hh_inc1 * db["HH_INC_ABS"] + b_carav1 * db["CAR_AVAILABILITY"]
    + b_com * db["COMMUTE"] + b_shop * db["SHOPPING"] + b_bis * db["BUSINESS"] 
    + b_lei * db["LEISURE"]
    + asc_alt1
)
U_2 = (
    b_cost * db["TC2"] + b_time * db["TT2"] + b_hw * db["HW2"] + b_ch * db["CH2"] 
    + b_hh_inc2 * db["HH_INC_ABS"] + b_carav2 * db["CAR_AVAILABILITY"]
    + asc_alt2
)

# Associate utility functions with the list
# U = [U_1, U_2]

U = ResLogitLayer([U_1, U_2], W1, W2).output

# Associate the availability conditions with the alternatives
av = [db["AV1"], db["AV2"]]

# rll = ResLogitLayer(U, W1, W2)
model = MNLogit(U, av, database=db, name="mymodel")
model.add_params(locals())


# train function
model = cmt.train(model, database=db, optimizer=Adam, batch_size=100, lr_init=0.01, max_epoch=999, notebook=True)

with open("myModel.pkl", "rb") as f:
    model = pickle.load(f)

result = Results(model, db, show_weights=True)

Building model...
dataset: model_average_sp (2790)
batch size: 100
batches per epoch: 27
validation frequency: 27

Training model...


Loglikelihood:  -1966.516153  Score: 0.502

Epoch    0/26973:   0%|          | 0.00/27.0k [00:00<?, ?it/s]

Optimization complete with accuracy of 79.606% with maximum loglikelihood reached @ epoch 999.

Results for model: mymodel
Build time: 00:00:12
Estimation time: 00:00:56
Estimation rate: 17.801 epochs/s
Seed value: 999
Number of Beta parameters: 11
Tensor size: 40
Sample size: 2790
Excluded data: 0
Init loglikelihood: -1966.516
Final loglikelihood: -1282.160
Final loglikelihood reached at: epoch 999
Likelihood ratio test: 1368.712
Accuracy: 79.606%
Rho square: 0.348
Rho bar square: 0.322
Akaike Information Criterion: 2666.32
Bayesian Information Criterion: 2968.94
Final gradient norm: 0.012

Statistical Analysis:
              Value   Std err     t-test   p-value Rob. Std err  Rob. t-test Rob. p-value
asc_alt1        0.0         -          -         -            -            -            -
asc_alt2   0.241534  0.631401   0.382537  0.702063     0.600677     0.402103     0.687608
b_bis      0.234032  0.641619   0.364753  0.715296     0.593899     0.394061     0.693536
b_carav1  -0.125164

In [5]:
result = Results(model, db, show_weights=True)


Results for model: mymodel
Build time: 00:00:12
Estimation time: 00:00:56
Estimation rate: 17.801 epochs/s
Seed value: 999
Number of Beta parameters: 11
Tensor size: 40
Sample size: 2790
Excluded data: 0
Init loglikelihood: -1966.516
Final loglikelihood: -1282.160
Final loglikelihood reached at: epoch 999
Likelihood ratio test: 1368.712
Accuracy: 79.606%
Rho square: 0.348
Rho bar square: 0.322
Akaike Information Criterion: 2666.32
Bayesian Information Criterion: 2968.94
Final gradient norm: 0.012

Statistical Analysis:
              Value   Std err     t-test   p-value Rob. Std err  Rob. t-test Rob. p-value
asc_alt1        0.0         -          -         -            -            -            -
asc_alt2   0.241534  0.631401   0.382537  0.702063     0.600677     0.402103     0.687608
b_bis      0.234032  0.641619   0.364753  0.715296     0.593899     0.394061     0.693536
b_carav1  -0.125164  0.047996   -2.60778  0.009113     0.000461  -271.636353          0.0
b_carav2        0.0      

In [6]:
type(db.TC1)

biogeme.expressions.Variable